In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame

# 1. 데이터 만들기 

In [2]:
# 데이터 불러오기
def load_all():
    train_activity = pd.read_csv('./train_activity.csv')
    train_combat = pd.read_csv('./train_combat.csv')
    train_payment = pd.read_csv('./train_payment.csv')
    train_trade = pd.read_csv('./train_trade.csv')
    train_pledge = pd.read_csv('./train_pledge.csv')
    train_label = pd.read_csv('./train_label_add.csv')
    return train_activity, train_combat, train_payment, train_trade, train_pledge, train_label

In [3]:
train_activity, train_combat, train_payment, train_trade, train_pledge, train_label = load_all()

In [4]:
# 1단계: char_id를 압축으로 인한 정보 손실 보강할 변수 추가

# 2단계: 모든 데이터를 acc_id, day 기준으로 합치기 (char_id는 압축)

# 2-1) 압축하기
activity = train_activity.groupby(['acc_id','day']).sum().reset_index().drop('char_id',axis=1)
combat = train_combat.groupby(['acc_id','day']).sum().reset_index().drop('char_id',axis=1)
payment = train_payment.groupby(['acc_id','day']).sum().reset_index()
pledge = train_pledge.groupby(['acc_id','day']).sum().reset_index().drop('char_id',axis=1)

In [5]:
#trade

# 주는 유저
train_trade_source = train_trade[['day', 'time', 'type', 'server', 'source_acc_id', 
                            'source_char_id', 'item_type', 'item_amount','item_price']]
# 받는 유저
train_trade_target = train_trade[['day', 'time', 'type', 'server', 'target_acc_id', 
                            'target_char_id', 'item_type', 'item_amount','item_price']]

train_trade_source = train_trade_source.rename(columns = {'source_acc_id': 'acc_id', 'source_char_id':'char_id',
                                                         'item_amount': 'item_amount_s', 'item_price': 'item_price_s'})
train_trade_target = train_trade_target.rename(columns = {'target_acc_id': 'acc_id', 'target_char_id':'char_id',
                                                         'item_amount': 'item_amount_t', 'item_price': 'item_price_t'})

# '시간' 컬럼을 그룹별로 합침(time, item_type, server 제거됨  )
train_trade_source = train_trade_source.groupby(['day','acc_id','char_id']).sum()
train_trade_source = train_trade_source.reset_index()

train_trade_target = train_trade_target.groupby(['day','acc_id','char_id']).sum()
train_trade_target = train_trade_target.reset_index()
train_trade_target['item_price_t'] = train_trade_target['item_price_t']*(-1)

standard = activity[['acc_id','day','playtime']]
source = pd.merge(standard, train_trade_source , on=['day','acc_id'], how = 'outer').dropna(subset = ['playtime']).drop(['char_id','playtime'],axis=1)
target = pd.merge(standard, train_trade_target , on=['day','acc_id'], how = 'outer').dropna(subset = ['playtime']).drop(['char_id','playtime'],axis=1)


In [6]:
trade = pd.merge(source, target , on=['day','acc_id'], how = 'outer')

In [7]:
trade = trade.groupby(['acc_id','day']).sum().reset_index()

In [8]:
# 3단계: 데이터 합치기
data_ac = pd.merge(activity, combat, on = ['acc_id','day'])

# 현금 결제만 하고 플레이는 안 한 경우가 있어서, 인스턴스가 945628 -> 946632로 증가 (결측값을 0으로 채움)
data_acp = pd.merge(data_ac, payment, on = ['acc_id','day'], how='outer').fillna(0) 

data_acpl = pd.merge(data_acp, pledge, on = ['acc_id','day'], how='outer').fillna(0)
data_acplt = pd.merge(data_acpl, trade, on = ['acc_id','day'], how='outer').dropna(subset = ['playtime']).fillna(0)

In [9]:
data_acplt.shape

(946632, 42)

In [11]:
# 28*40000으로 데이터프레임 만들기

# 실제 접속일  데이터
real_day = data_acplt[['acc_id','day', 'playtime', 'npc_kill', 'solo_exp', 'party_exp',
       'quest_exp', 'rich_monster', 'death', 'revive', 'exp_recovery',
       'fishing', 'private_shop', 'game_money_change', 'enchant_count',
       'class', 'level', 'pledge_cnt', 'random_attacker_cnt_x',
       'random_defender_cnt_x', 'temp_cnt_x', 'same_pledge_cnt_x', 'etc_cnt_x',
       'num_opponent', 'amount_spent','play_char_cnt',
       'combat_char_cnt', 'pledge_combat_cnt', 'random_attacker_cnt_y',
       'random_defender_cnt_y', 'same_pledge_cnt_y', 'temp_cnt_y', 'etc_cnt_y',
       'combat_play_time', 'non_combat_play_time', 'item_amount_s',
       'item_price_s', 'item_amount_t', 'item_price_t']]
real_day = real_day.groupby(['acc_id','day']).sum().reset_index()

# 전체 일자 데이터 (40000*28)
day = np.zeros(shape=(40000*28,2))

# acc_id 40000개 추출
acc_id = train_activity[['acc_id','playtime']]
acc_id = acc_id.groupby('acc_id').sum().reset_index()
acc_id = acc_id['acc_id']
acc_id = acc_id.values

# day에 acc_id와 1~28일 입력 후 데이터프레임으로 변환
for i in range(0,40000):
    for j in range(0,28):
        n = 28*i+j
        day[n][0] = acc_id[i]
        day[n][1] = j+1
day_df = DataFrame(day, columns=['acc_id','day'])

# 만든 데이터프레임에 실제 접속일 데이터 merge
stop_count_df = pd.merge(day_df, real_day,  on=['acc_id','day'],how='outer').fillna(0)
# 접속하지 않은 날은 0, 접속일은 1로 표시
#stop_count_df['login'] = np.where(stop_count_df['playtime'].isnull(), 0, 1)
#stop_count_df['stop'] = 0
#stop_count_df['day'] = np.where(stop_count_df['playtime'].isnull(), 0, stop_count_df['day'])

In [25]:
real_day

,acc_id,day,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,...,random_defender_cnt_y,same_pledge_cnt_y,temp_cnt_y,etc_cnt_y,combat_play_time,non_combat_play_time,item_amount_s,item_price_s,item_amount_t,item_price_t
0,2,1,2.703457,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.732562e-06,3.456524,0.000000e+00,0.000000
1,2,2,3.363522,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.116080e-01,0.201926,0.000000e+00,0.000000
2,2,3,3.370544,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.430650e-07,1.532037,0.000000e+00,0.000000
3,2,4,3.370544,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
4,2,5,3.370544,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.396984e-08,0.075984,0.000000e+00,0.000000
5,2,6,3.370544,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.396984e-08,0.051804,0.000000e+00,0.000000
6,2,7,3.370544,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.198492e-06,1.196498,0.000000e+00,0.000000
7,2,8,2.768995,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
8,2,9,3.361181,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,4.793968e-08,-0.013612
9,2,10,3.370544,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.475627e-06,0.466400,0.000000e+00,0.000000


In [26]:
stop_count_df.columns

Index(['acc_id', 'day', 'playtime', 'npc_kill', 'solo_exp', 'party_exp',
       'quest_exp', 'rich_monster', 'death', 'revive', 'exp_recovery',
       'fishing', 'private_shop', 'game_money_change', 'enchant_count',
       'class', 'level', 'pledge_cnt', 'random_attacker_cnt_x',
       'random_defender_cnt_x', 'temp_cnt_x', 'same_pledge_cnt_x', 'etc_cnt_x',
       'num_opponent', 'amount_spent', 'play_char_cnt', 'combat_char_cnt',
       'pledge_combat_cnt', 'random_attacker_cnt_y', 'random_defender_cnt_y',
       'same_pledge_cnt_y', 'temp_cnt_y', 'etc_cnt_y', 'combat_play_time',
       'non_combat_play_time', 'item_amount_s', 'item_price_s',
       'item_amount_t', 'item_price_t'],
      dtype='object')

In [117]:
# 정규화
stop_count_acc = stop_count_df[['acc_id','day']]
stop_count_col = stop_count_df[['playtime', 'npc_kill', 'solo_exp', 'party_exp',
       'quest_exp', 'rich_monster', 'death', 'revive', 'exp_recovery',
       'fishing', 'private_shop', 'game_money_change', 'enchant_count',
       'class', 'level', 'pledge_cnt', 'random_attacker_cnt_x',
       'random_defender_cnt_x', 'temp_cnt_x', 'same_pledge_cnt_x', 'etc_cnt_x',
       'num_opponent', 'amount_spent', 'play_char_cnt', 'combat_char_cnt',
       'pledge_combat_cnt', 'random_attacker_cnt_y', 'random_defender_cnt_y',
       'same_pledge_cnt_y', 'temp_cnt_y', 'etc_cnt_y', 'combat_play_time',
       'non_combat_play_time']]


min1 = np.min(stop_count_col, 0) # 각 열마다 최소값 
max1 = np.max(stop_count_col, 0) # 각 열마다 최대값 
stop_count_col_pre = (stop_count_col-min1)/(max1-min1) # 정규화: (X-min) / (max - min)
stop_count_col_pre.head() # 정규화된 데이터

,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,exp_recovery,fishing,...,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt_y,random_defender_cnt_y,same_pledge_cnt_y,temp_cnt_y,etc_cnt_y,combat_play_time,non_combat_play_time
0,0.734266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.913541,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.915448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.915448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.915448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
# acc,day 데이터와 정규화된 데이터 합치기
data_final = pd.concat([stop_count_acc,stop_count_col_pre], axis=1)

In [120]:
# shape=(40000,28,39)인 데이터 틀 만듦
day = np.zeros(shape=(40000,28,35))
for i in range(0,40000):
    for j in range(0,28):
        day[i][j][0] = acc_id[i]
        day[i][j][1] = j+1

In [121]:
# 40000개의 3차원 인덱스에 acc_id별로 28*39 넣기
for i in range(0,40000):
    acc = acc_id[i]
    day[i] = data_final[data_final['acc_id']==acc]

In [113]:
day[2]

array([[8.00000000e+00, 1.00000000e+00, 1.25874126e-01, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [8.00000000e+00, 2.00000000e+00, 1.51303242e-01, ...,
        0.00000000e+00, 6.66087962e-07, 1.00000000e+00],
       [8.00000000e+00, 3.00000000e+00, 3.21042594e-01, ...,
        0.00000000e+00, 6.66087962e-04, 1.00000000e+00],
       ...,
       [8.00000000e+00, 2.60000000e+01, 5.46726001e-02, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [8.00000000e+00, 2.70000000e+01, 8.51875397e-02, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [8.00000000e+00, 2.80000000e+01, 3.64907819e-01, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [122]:
# 'acc_id', 'day' 행을 제거
day_final = day[:,:,2:]

In [123]:
day_final.shape

(40000, 28, 33)

In [15]:
# label 만들기
# 모든 데이터를 합친 data_acplt와 acc_id를 똑같이 정렬
train_label1 = train_label.sort_values('acc_id').reset_index()
# 클래스 레이블 별로 분리
survival_time = train_label1['survival_time'].values.reshape(40000,1)
amount_spent = train_label1['amount_spent'].values.reshape(40000,1)
secession = train_label1['secession'].values.reshape(40000,1)
total_spent = train_label1['total_spent'].values.reshape(40000,1)

# 2. LSTM 적용 

In [124]:
# 데이터 분리
train_x = day_final[:28000]
train_y = survival_time[:28000]
test_x = day_final[28000:]
test_y = survival_time[28000:]

In [133]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout


# define model
model = Sequential()
model.add(LSTM(128, return_sequences = True,input_shape=(28, 33)))
model.add(LSTM(128))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=['acc']) 

model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 28, 128)           82944     
_________________________________________________________________
lstm_21 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_23 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_24 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 65        
Total params: 239,361
Trainable params: 239,361
Non-trainable params: 0
_________________________________________________________________


In [134]:
# fit model
model.fit(train_x, train_y, epochs=200, batch_size=128)

Epoch 1/200
28000/28000 [==============================] - 24s 864us/step - loss: 665.3145 - acc: 0.0048
Epoch 2/200
28000/28000 [==============================] - 23s 816us/step - loss: 517.2074 - acc: 0.0081
Epoch 3/200
28000/28000 [==============================] - 23s 832us/step - loss: 445.0353 - acc: 0.0166
Epoch 4/200
28000/28000 [==============================] - 23s 827us/step - loss: 423.6312 - acc: 0.0170
Epoch 5/200
28000/28000 [==============================] - 23s 830us/step - loss: 419.5769 - acc: 0.0168
Epoch 6/200
28000/28000 [==============================] - 23s 826us/step - loss: 410.2455 - acc: 0.0169
Epoch 7/200
28000/28000 [==============================] - 23s 821us/step - loss: 405.0124 - acc: 0.0174
Epoch 8/200
28000/28000 [==============================] - 23s 822us/step - loss: 404.2008 - acc: 0.0178
Epoch 9/200
23296/28000 [=======================>......] - ETA: 3s - loss: 400.8220 - acc: 0.0176

KeyboardInterrupt: 

결론: LSTM은 적합한 모델이 아니다.

이유 


1) 유저마다 접속일수가 다르기 때문에 '0'이 많은 sparse data가 된다.

2) 클래스 레이블이 구간별로 수가 매우 적기 때문에 데이터가 모자라다. 